### 커스텀 LLM 클래스 정의 

In [ ]:
from langchain.llms.base import LLM
from typing import Optional, List, Dict
import requests

class OllamaLLM(LLM):
    api_url: str = "http://localhost:11434/api/generate" 
    model_name: str = "llama3.2:3B" 

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        
        payload = {
            "model": self.model_name,
            "prompt": prompt,
            "stream": False
        }
        response = requests.post(self.api_url, json=payload)
        response_data = response.json()
        return response_data.get("response", "No response received.")

    @property
    def _identifying_params(self) -> Dict[str, str]:
        return {"model_name": self.model_name}
    
    @property
    def _llm_type(self) -> str:
        return "ollama"

ollama_llm = OllamaLLM()

prompt = "Why is the sky blue?"

response = ollama_llm.invoke(prompt)
print(f"Response: {response}")

In [ ]:
from langchain_core.prompts import PromptTemplate

# Instantiation using from_template (recommended)
prompt = PromptTemplate.from_template("Say {foo}")
prompt.format(foo="bar")

# Instantiation using initializer
#prompt = PromptTemplate(template="Say {foo}")

In [ ]:
prompt

In [ ]:
prompt = PromptTemplate(template = """
Hello, my name is {name}. I work as a {profession} with {experience} years of experience. 
I specialize in {specialization} and have a deep interest in {interests}. 
Throughout my career, I've been involved in projects like {projects}, which helped me develop strong skills in {skills}.

In my free time, I enjoy {hobbies}, as it allows me to relax and recharge. 
My goal is to {goal}, and I'm constantly looking for opportunities to grow and learn more in this field.
People often describe me as {character_traits}, which I believe helps me succeed in my career.

Thank you for giving me the opportunity to introduce myself.
""")

In [ ]:
prompt

### Prompt Template 사용방법

In [ ]:
profile_dict = {
    "name":"Alice",
    "profession":"Data Scientist",
    "experience":"5",
    "specialization":"machine learning and data analysis",
    "interests":"AI ethics and advanced analytics",
    "projects":"predictive analytics for sales, customer segmentation models",
    "skills":"data modeling, statistical analysis, and machine learning",
    "hobbies":"reading science fiction and hiking",
    "goal":"make impactful contributions in the AI field",
    "character_traits":"curious, detail-oriented, and collaborative"
}

In [ ]:
profile_dict['hobbies']

In [ ]:
print(prompt.format(**profile_dict))

In [ ]:
from langchain.prompts import PromptTemplate

# 자기소개 프롬프트 템플릿 생성
template_text = """
Hello, my name is {name}. I work as a {profession} with {experience} years of experience. 
I specialize in {specialization} and have a deep interest in {interests}. 
Throughout my career, I've been involved in projects like {projects}, which helped me develop strong skills in {skills}.

In my free time, I enjoy {hobbies}, as it allows me to relax and recharge. 
My goal is to {goal}, and I'm constantly looking for opportunities to grow and learn more in this field.
People often describe me as {character_traits}, which I believe helps me succeed in my career.

Thank you for giving me the opportunity to introduce myself.
"""

# 템플릿 생성
prompt = PromptTemplate.from_template(template_text)

# format 메서드를 사용해 변수 값 채우기
result = prompt.format(
    aa
)

print(result)

### Chains 템플릿 사용방법

In [ ]:
from langchain.chains import SequentialChain
from langchain.prompts import PromptTemplate

# 첫 번째 프롬프트: 직업을 생성
generator_prompt = PromptTemplate.from_template(
    "Randomly create a job name."
)

# 두 번째 프롬프트: 직업을 위한 스킬 목록 생성
question_prompt = PromptTemplate.from_template(
    "List the top 5 key skills needed for a {job}."
)

# 세 번째 프롬프트: 간략하게 요약
answer_prompt = PromptTemplate.from_template(
    "Summarize these key skills for a {job} in one short sentence: {skills}. "
    "Explain why these skills are critical in a concise way."
)

# 첫 번째 LLM 체인 생성
generator_chain = LLMChain(prompt=generator_prompt, llm=ollama_llm, output_key="job")

# 두 번째 LLM 체인 생성
question_chain = LLMChain(prompt=question_prompt, llm=ollama_llm, output_key="skills")

# 세 번째 LLM 체인 생성
answer_chain = LLMChain(prompt=answer_prompt, llm=ollama_llm, output_key="summary")

# SequentialChain을 이용하여 세 체인 연결
chain = SequentialChain(
    chains=[generator_chain, question_chain, answer_chain],
    input_variables=[],
    output_variables=["summary"]
)

# 체인 실행
result = chain.invoke({})
print(result)

In [ ]:
generator_chain.invoke({})

In [ ]:
question_chain.invoke({'job': '"Interdimensional Experience Coordinator"'})

In [ ]:
answer_chain.invoke({'job': '"Interdimensional Experience Coordinator"',
 'skills': "What a fascinating role! Here are the top 5 key skills needed for an Interdimensional Experience Coordinator:\n\n**1. Multiverse Navigation**: The ability to navigate and understand different dimensions, realities, or planes of existence is crucial for this role. This requires knowledge of interdimensional travel methods, understanding of dimensional energies, and the capacity to adapt to new environments.\n\n**2. Interpersonal Communication**: Effective communication with beings from various dimensions, realities, and cultures is essential. The ability to bridge gaps in language, customs, and ideologies will enable the coordinator to facilitate smooth interactions and ensure successful experiences.\n\n**3. Experience Design and Facilitation**: As an Interdimensional Experience Coordinator, you'll design and lead immersive experiences that cater to diverse interdimensional travelers. This requires creativity, empathy, and problem-solving skills to tailor experiences that meet the unique needs of each guest.\n\n**4. Energy Management and Regulation**: Managing and regulating the flow of energy across dimensions is critical for maintaining stability and preventing unintended consequences. This involves understanding the intricacies of dimensional resonance, vibration frequency, and the impact of energy exchange on both individuals and the fabric of reality.\n\n**5. Temporal Awareness and Adaptability**: Interdimensional travel often requires flexibility in regards to temporal aspects. The ability to navigate multiple timelines, account for divergent outcomes, and adapt to changing circumstances will be essential for this role. This involves maintaining a deep understanding of temporal dynamics, being able to analyze and interpret temporal anomalies, and making decisions that minimize disruptions to the fabric of reality.\n\nWhile these skills are hypothetical, I hope this gives you an idea of what it might take to excel in this imaginative profession!"})['summary']

### agent 사용법

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key="api_key")

In [ ]:
from langchain.agents import tool
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaAPIWrapper()
wikipedia_tool = Tool(name="Wikipedia",
                      func=wikipedia.run,
                      description=""
                     )

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


#get_word_length.invoke("abc")
tools = [get_word_length, wikipedia_tool]

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "try to use wikipedia when needed",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm_with_tools = llm.bind_tools(tools)

from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

list(agent_executor.stream({"input": "How many letters in the longest word in English?"}))




### memory 사용법

In [ ]:
from langchain.agents import tool
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaAPIWrapper()
wikipedia_tool = Tool(name="Wikipedia",
                      func=wikipedia.run,
                      description=""
                     )

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


#get_word_length.invoke("abc")
tools = [get_word_length, wikipedia_tool]

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "try to use wikipedia when needed",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

llm_with_tools = llm.bind_tools(tools)

from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [ ]:
input1 = "How many letters in the longest word in English?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})

In [ ]:
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)

In [ ]:
chat_history

In [ ]:
input2 = "What is the second?"
agent_executor.invoke({"input": input2, "chat_history": chat_history})